# Projet  **<span style="color: #CC146C">Quiz generator </span>** 💡 - Python pour le data-scientist

#### Auteurs : Adrien Servière, Mélissa Tamine.

L'objectif de ce notebook est de présenter le projet que nous avons effectué dans le cadre de l'unité d'enseignement **Python pour le data-scientist** dispensée à l'ENSAE. Ce projet a été élaboré de manière libre et comporte, comme attendu, un **jeu de données** récupéré et traité, une partie **visualisation** et une partie **modélisation**. 

# Problématique

Notre projet s'articule autour de la problématique suivante : **Comment créer un système capable de générer un quiz (plusieurs paires de question/réponse) sur un thème précis ?**

Dans la mesure où l'objectif principal d'un quiz est d’évaluer les connaissances d’un participant, il nous a semblé qu'un tel système pourrait s'avérer très utile à des enseignants afin de tester de manière ludique les acquis de leurs élèves par exemple. 

C'est pourquoi nous avons modélisé la structure suivante afin que le système créé puisse répondre au problème :

![framework](./data/images/framework.png "Structure du système implémenté")

La structure est divisée en deux parties distinctes : 
1. Une partie **traitement des données** qui a principalement consisté à extraire et indexer dans ElasticSearch la base de données Wikipédia sur laquelle le modèle se fonde.
2. Une partie **modélisation** fondée sur la mise en place d'une *pipeline* formée de plusieurs outils de traitement du langage et modèles de langages.

# Installations et recommandations préalables

Avant d'exécuter veuillez procéder aux installations de modules nécessaires au bon fonctionnement du code en exécutant la cellule ci-dessous. 

In [19]:
!pip install elasticsearch
!pip install wordcloud
#!pip install torch

     |████████████████████████████████| 385 kB 1.6 MB/s            
     |████████████████████████████████| 363 kB 1.6 MB/s            


De même, nous vous demanderons d'exécuter les cellules de ce notebook au sein d'un **espace de travail muni d'un service ElasticSearch préalablement exécuté** afin que la partie indexation puisse fonctionner. Nous vous conseillons d'utiliser **SSP Cloud** car la technologie ElasticSearch y est disponible.

# Importation des modules utiles

In [20]:
import os
import sys
import seaborn
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from src.models.bm25_retriever import *
from src.scripts.wikipedia_indexing import *

# Récupération et traitement des données 🗂️

## Extraction des données textuelles provenant de l'encyclopédie Wikipédia

La première étape de notre projet a été la **récupération** et le **traitement des données**. Dans notre cas, nous avons fait le choix de récupérer des données textuelles provenant d'articles de l'encyclopédie en ligne **Wikipédia**. 

Pour des raisons de volume, nous avons extrait les données brutes de la version en anglais simple de Wikipédia (en anglais : *Simple English Wikipedia*). Il s'agit d'une encyclopédie spécialement fondée pour « des étudiants, des enfants ou des adultes ayant des difficultés de compréhension et pour ceux qui souhaiteraient apprendre l'anglais ». En novembre 2021, date à laquelle nous avons extrait les données brutes, le site contenait plus de **200 000 pages** différentes.

Afin d'extraire ces données brutes et de les convertir en données textuelles pouvant être exploitées, nous avons utilisé l'outil ```Wikiextractor``` (https://attardi.github.io/wikiextractor/) de la manière suivante dans un terminal :
```
>>> wget "http://download.wikimedia.org/simplewiki/latest/simplewiki-latest-pages-articles.xml.bz2"

>>> python -m wikiextractor.WikiExtractor -o "./quiz-generator/data/wikipedia/" --json --processes 12 "quiz-generator/data/wikipedia/simplewiki-latest-pages-articles.xml.bz2"

>>> rm "./quiz-generator/data/wikipedia/simplewiki-latest-pages-articles.xml.bz2"
```

Ces commandes nous ont permis d'obtenir les plus de 200 000 pages de données textuelles sous la forme de plusieurs fichiers .txt formatés comme des données au format JSON que vous trouverez dans le dossier *data* du projet.

## Indexation des données textuelles dans ElasticSearch

Une fois les données récupérées, il a ensuite fallu les traiter. Le traitement a principalement consisté à indexer ces données textuelles dans *ElasticSearch*. 

ElasticSearch c’est un logiciel qui fournit un moteur de recherche installé sur un serveur (dans notre cas le serveur SSP Cloud) qu’il est possible de requêter depuis un client (ce Notebook en l'occurence). C’est un moteur de recherche très performant, puissant et flexible sur données textuelles. L'objectif de l'utilisation d'un tel outil est de trouver, dans un corpus de grande dimension, un certain texte. **Dans notre cas, il s'agit de trouver les textes les plus pertinents sur un thème donné parmi l'ensemble des données textuelles comprises dans les 200 000 pages de données disponibles.**

Nous utilisons la librairie python ```elasticsearch``` pour dialoguer avec notre moteur de recherche Elastic. La ligne de code ci-dessous permet d'établir la connexion avec le cluster Elastic que vous avez dû lancer dans votre session SSP Cloud lors de la phase de recommandation. 

In [21]:
es = set_es_client()

Maintenant que la connexion est établie, nous pouvons passer à l'étape **d'indexation**. Cette étape consiste à envoyer les documents parmi lesquels nous souhaitons chercher des echos pertinents dans notre elastic. Un index est donc une collection de document. Dans notre cas, les documents sont les paragraphes qui composent les articles du *Simple English Wikipedia*. 

**Remarque :** L'exécution de la ligne suivante qui permet l'indexation est relativement longue, veuillez compter environ 7 minutes.

In [40]:
#run_indexing(client=es, args=fill_default_args())

## Exécution d'une première requête test

Maintenant que l'étape d'indexation est finalisée, il est désormais possible de lancer notre première **requête** c'est à dire de chercher les documents les plus pertinents à propos d'un thème (un certain mot). 

Pour cela, nous utilisons l'algorithme BM25 que nous avons implémenté et qui utilise simplement la méthode interne de pondération des mots utilisée par ElasticSearch. La pertinence d’un mot pour notre recherche est construite sur une variante de la **TF-IDF**, considérant qu’un terme est pertinent s’il est souvent présent dans le document (Term Frequency) alors qu’il est peu fréquent dans les autres document (inverse document frequency).

In [23]:
bm25 = BM25Retriever(client=es)

Nous décidons par exemple de chercher les documents c'est à dire les paragraphes inclus dans le *Simple English Wikipedia* les plus pertinents traitant du philosophe **Emmanuel Kant**.

In [38]:
contexts = bm25.retrieve(query='kant')
text_contexts = [context.text for context in contexts]

In [39]:
nl = ' \n ------ \n'
print(f"{nl}{nl.join(text_contexts)}")

 
 ------ 
In the early 20th century Kant's ideas were very influential on one group of German philosophers. They became known as the new-Kantians. One of them, Windelband, said, "every philosophy before Kant poured into Kant, and every philosophy after Kant pours from Kant". 
 ------ 
Some scholars like to include Kant as one of the German idealists, but Kant himself did not belong to that group. 
 ------ 
The term was coined by Immanuel Kant. Kant said that moral values had to be unconditional, categorical; he wrote a categorical imperative. 
 ------ 
The most-known work of Kant is the book "Critique of Pure Reason" ("Kritik der reinen Vernunft") that Kant published in 1781. Kant called his way of thought "critique", not philosophy. Kant said that critique was a preparation for establishment of real philosophy. According to Kant, people should know what human reason can do and which limits it has. In the "Critique of Pure Reason" Kant wrote several limits of human reason, to both fee

# Visualisation des données 📊

# Modélisation 🔍